In [1]:
import boto3
import pandas as pd
from ete3 import NCBITaxa

In [3]:
s3 = boto3.resource('s3')
client = boto3.client('s3')
bucket = s3.Bucket("czbiohub-mosquito")

In [45]:
bucket_name = "czbiohub-mosquito"
list_of_folders = client.list_objects(Bucket=bucket_name, Prefix="contigs/")
filenames = pd.DataFrame(["s3://"+bucket_name+"/"+x["Key"] for x in list_of_folders["Contents"] if "blast_nt" in x["Key"]], columns=["blast_nt"])
filenames = filenames.assign(filtered_blast_nt=filenames["blast_nt"].apply(lambda x: x.replace("contigs", "contig_quality").replace(".m9", "_filtered.m9")))
filenames = filenames.assign(lca_nt=filenames["filtered_blast_nt"].apply(lambda x: x.replace("blast_nt_filtered.m9", "lca_nt.m9")))


The nt hits of contigs from each sample are filtered with ident_cutoff=0.5 and align_len_cutoff=0.2

In [97]:
commands = filenames.apply(lambda x: "python lca_analysis.py --blast_type nt --fpath "+x.iloc[0]+\
                " --filtered_blast_path "+x.iloc[1]+" --outpath "+x.iloc[2]+\
                " --ident_cutoff 0.5 --align_len_cutoff 0.5", axis=1)


In [102]:
commands.to_csv("lca_analysis_commands", index=False)

In [ ]:
%%bash
parallel -a lca_analysis_commands -j 16

